In [18]:
import os
import re
import numpy as np
from collections import defaultdict
from math import log

In [19]:
# Preprocessing function
def preprocess(text):
    return re.findall(r'\b\w+\b', text.lower())

# Load documents
def load_documents(folder_path):
    docs = {}
    for filename in os.listdir(folder_path):
        if filename.endswith('.txt'):
            with open(os.path.join(folder_path, filename), 'r') as file:
                docs[filename] = preprocess(file.read())
    return docs

In [25]:
# Load queries
def load_queries(query_file_path):
    with open(query_file_path, 'r') as file:
        return [line.strip() for line in file.readlines()]

# Compute term frequencies and document frequencies
def compute_statistics(docs):
    doc_count = len(docs)
    term_doc_freq = defaultdict(int)
    term_freq = defaultdict(lambda: defaultdict(int))

    for doc_id, words in docs.items():
        word_set = set(words)
        for word in words:
            term_freq[doc_id][word] += 1
        for word in word_set:
            term_doc_freq[word] += 1

    return term_freq, term_doc_freq, doc_count

# Compute relevance probabilities using BIM
def compute_relevance_prob(query, term_freq, term_doc_freq, doc_count):
    scores = {}
    for doc_id in term_freq:
        score = 1.0
        for term in query:
            tf = term_freq[doc_id].get(term, 0)
            df = term_doc_freq.get(term, 0)
            p_term_given_relevant = (tf + 1) / (sum(term_freq[doc_id].values()) + len(term_doc_freq))
            p_term_given_not_relevant = (df + 1) / (doc_count - df + len(term_doc_freq))
            score *= (p_term_given_relevant / p_term_given_not_relevant)
        scores[doc_id] = score
    return scores

# Main retrieval function
def retrieve_documents(folder_path, query_file_path):
    docs = load_documents(folder_path)
    queries = load_queries(query_file_path)

    term_freq, term_doc_freq, doc_count = compute_statistics(docs)

    for query in queries:
        query_terms = preprocess(query)
        scores = compute_relevance_prob(query_terms, term_freq, term_doc_freq, doc_count)
        ranked_docs = sorted(scores.items(), key=lambda item: item[1], reverse=True)
        print(f"Query: {query}")
        for doc_id, score in ranked_docs:
            print(f"Document: {doc_id}, Score: {score:.4f}")
        print()

# Example usage
folder_path = '/content/drive/MyDrive/Tech400_animals'
query_file_path = '/content/drive/MyDrive/queries_results/queries.txt'
retrieve_documents(folder_path, query_file_path)

Query: to
Document: Pandas.txt, Score: 0.6365
Document: Koalas.txt, Score: 0.6014
Document: Camels.txt, Score: 0.5945
Document: Horses.txt, Score: 0.5160
Document: Deers.txt, Score: 0.4738
Document: Dogs.txt, Score: 0.4393
Document: Lions.txt, Score: 0.4367
Document: Kangaroos.txt, Score: 0.4355
Document: Hyenas.txt, Score: 0.3998
Document: Chimpanzees.txt, Score: 0.3994
Document: Tiger.txt, Score: 0.3978
Document: Rhinos.txt, Score: 0.3978
Document: Zebras.txt, Score: 0.3961
Document: Squirrels.txt, Score: 0.3592
Document: Cows.txt, Score: 0.3563
Document: Ostrich.txt, Score: 0.3555
Document: Cats.txt, Score: 0.3202
Document: Bears.txt, Score: 0.2819
Document: Elephants.txt, Score: 0.2773
Document: Gorillas.txt, Score: 0.2404

Query: describe the social behavior
Document: Elephants.txt, Score: 0.0374
Document: Hyenas.txt, Score: 0.0170
Document: Chimpanzees.txt, Score: 0.0161
Document: Zebras.txt, Score: 0.0109
Document: Lions.txt, Score: 0.0077
Document: Gorillas.txt, Score: 0.0073
D

In [26]:
import random

import random

# Function to assign random relevance scores
def assign_random_relevance(queries, documents, relevance_scale=(0, 1)):
    relevance_scores = {}

    for query in queries:
        relevance_scores[query] = {}  # Use the query string directly, no tuple or list
        for doc in documents:
            # Assign a random relevance score between relevance_scale (0 and 1 by default)
            relevance_scores[query][doc] = random.randint(relevance_scale[0], relevance_scale[1])

    return relevance_scores

In [27]:
# Function to save relevance scores to a file
def save_relevance_scores_to_file(relevance_scores, output_file):
    with open(output_file, 'w') as f:
        for query, doc_scores in relevance_scores.items():
            for doc, score in doc_scores.items():
                f.write(f"{query},{doc},{score}\n")  # Write in the format query,document,score


# Example usage:
folder_path = '/content/drive/MyDrive/Tech400_animals'
query_file_path = '/content/drive/MyDrive/queries_results/queries.txt'
# Load documents and queries
documents = load_documents(folder_path)  # This returns a dictionary of doc_id -> content
queries = load_queries(query_file_path)  # This returns a list of queries

# Randomly assign relevance scores (0 for irrelevant, 1 for relevant)
random_relevance_scores = assign_random_relevance(queries, documents.keys())

# Save the relevance scores to query_relevance_score.txt
output_file = '/content/drive/MyDrive/queries_results/query_relevance_score.txt'
save_relevance_scores_to_file(random_relevance_scores, output_file)